In [6]:
#Oppdatert
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

# Sett riktig mappe for lagring (opp ett nivå)
data_dir = "../data"
os.makedirs(data_dir, exist_ok=True)

API = "2a430097-ac9b-44e4-995e-a02afb9795b2"
SOURCE = "SN18700"
BASE_URL = "https://frost.met.no/observations/v0.jsonld"

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
delta = timedelta(days=30)

# Elementer som hentes i batcher (i én spørring)
elements_batch = {
    "mean(air_temperature P1D)": "Temperatur (°C)",
    "mean(air_pressure_at_sea_level P1D)": "Lufttrykk (hPa)",
    "sum(precipitation_amount P1D)": "Nedbør (mm)"
}

# Elementer som må hentes separat (på grunn av API-begrensninger)
elements_single = {
    "mean(relative_humidity P1D)": "Relativ fuktighet (%)",
    "mean(cloud_area_fraction P1D)": "Skydekke (oktas)"
}

force_update = input("Vil du hente nye data fra Frost API? (ja/nei): ").strip().lower()

# Funksjon for å hente data i batcher
def fetch_batch_elements():
    dataframes = {label: pd.DataFrame(columns=["Dato", label]) for label in elements_batch.values()}

    if all(os.path.exists(os.path.join(data_dir, f"{label}.csv")) for label in elements_batch.values()) and force_update != "ja":
        print("Bruker eksisterende filer for temperatur, trykk og nedbør.")
        return

    print("Henter temperatur, trykk og nedbør...")

    current_date = start_date
    while current_date <= end_date:
        batch_start = current_date.strftime("%Y-%m-%d")
        batch_end = min(current_date + delta, end_date).strftime("%Y-%m-%d")
        batch_reference_time = f"{batch_start}/{batch_end}"

        params = {
            "sources": SOURCE,
            "elements": ",".join(elements_batch.keys()),
            "referencetime": batch_reference_time
        }

        response = requests.get(BASE_URL, params=params, auth=(API, ""))
        
        if response.status_code == 200:
            raw_data = response.json().get("data", [])
            
            for entry in raw_data:
                dato = entry["referenceTime"].split("T")[0]
                if dato > end_date.strftime("%Y-%m-%d"):
                    continue
                for obs in entry["observations"]:
                    element_id = obs["elementId"]
                    verdi = obs["value"]
                    label = elements_batch[element_id]
                    df = dataframes[label]

                    df = pd.concat([df, pd.DataFrame([{"Dato": dato, label: verdi}])], ignore_index=True)
                    dataframes[label] = df
        else:
            print(f"Feil ved henting av data ({batch_reference_time}): {response.status_code}")
            print(response.text)

        current_date += delta

    for label, df in dataframes.items():
        df.drop_duplicates(subset="Dato", keep="last", inplace=True)
        df.sort_values("Dato", inplace=True)
        df.to_csv(os.path.join(data_dir, f"R_{label}.csv"), index=False, encoding="utf-8")
        print(f"Lagret til: {os.path.join(data_dir, f'R_{label}.csv')}")

# Funksjon for å hente enkeltstående elementer
def fetch_single_element(element_id, label):
    filename = os.path.join(data_dir, f"R_{label}.csv")
    if os.path.exists(filename) and force_update != "ja":
        print(f"Bruker eksisterende fil: {filename}")
        return

    print(f"Henter data for {label.lower()}...")
    params = {
        "sources": SOURCE,
        "elements": element_id,
        "referencetime": f"{start_date.strftime('%Y-%m-%d')}/{end_date.strftime('%Y-%m-%d')}"
    }

    response = requests.get(BASE_URL, params=params, auth=(API, ""))
    if response.status_code == 200:
        data = response.json().get("data", [])
        rows = [(d["referenceTime"].split("T")[0], d["observations"][0]["value"]) for d in data]
        pd.DataFrame(rows, columns=["Dato", label]).to_csv(filename, index=False, encoding="utf-8")
        print(f"{label} lagret som: {filename}")
    else:
        print(f"Feil ved henting av {label.lower()}: {response.status_code}\n{response.text}")

# Kjør begge deler
fetch_batch_elements()
for element_id, label in elements_single.items():
    fetch_single_element(element_id, label)


Henter temperatur, trykk og nedbør...


C:\Users\leahs\AppData\Local\Temp\ipykernel_29468\1160682549.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([{"Dato": dato, label: verdi}])], ignore_index=True)


Lagret til: ../data\R_Temperatur (°C).csv
Lagret til: ../data\R_Lufttrykk (hPa).csv
Lagret til: ../data\R_Nedbør (mm).csv
Henter data for relativ fuktighet (%)...
Relativ fuktighet (%) lagret som: ../data\R_Relativ fuktighet (%).csv
Henter data for skydekke (oktas)...
Skydekke (oktas) lagret som: ../data\R_Skydekke (oktas).csv
